- 1. Update config.yaml and Pramas.yaml

- 2. Update entity

In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Kidney_disease_classification\\research'

In [3]:
os.chdir('../')

In [12]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Kidney_disease_classification'

In [29]:
from dataclasses import dataclass
from pathlib import Path

In [30]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path

    params_image_size: list
    params_include_top: bool
    params_weight : str
    params_classes: int
    params_learning_rate: int
    
    

- 3. Update confiuration Manager

In [31]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml, create_directories
from src.cnn_classifier import logger

In [32]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model(self)-> PrepareBaseModelConfig:
        config=self.config.prepare_base_model
        
        prepare_base_model=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir), 
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),

            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_learning_rate=self.params.LEARNING_RATE
            )
        return prepare_base_model
        


- 4. Update components

In [33]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [34]:
class PrepareBaseModel: 
    def __init__(self, config: PrepareBaseModelConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight, 
            include_top=self.config.params_include_top
            )
        self.save_model(path=self.config.base_model_path, model=self.model)
        logger.info(f"Model save path {self.config.base_model_path} and model {self.model}")
        
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all: 
            for layer in model.layers: 
                model.trainable=False
            logger.info("Freeez all layer ")
        elif (freeze_till is not None) and (freeze_all > 0):
            for layer in model.layers[: -freeze_till]:
                model.trainable=False
            logger.info("Freeze cetrain part")

        
        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']

        )
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model=self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate

        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)




    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    
    
        

5. Update Pipeline

In [35]:
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e


[2023-10-10 16:47:00,963 - INFO, - common - yaml file : config\config.yaml loaded sucessfully]
[2023-10-10 16:47:00,968 - INFO, - common - yaml file : params.yaml loaded sucessfully]
[2023-10-10 16:47:00,971 - INFO, - common - Created directories at : artifacts]


[2023-10-10 16:47:03,538 - WARNING, - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2023-10-10 16:47:03,909 - INFO, - 2391830404 - Model save path artifacts\prepare_base_model\base_model.h5 and model <keras.engine.functional.Functional object at 0x000001CAD4598390>]
[2023-10-10 16:47:03,943 - INFO, - 2391830404 - Freeez all layer ]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 